Make a file of grid point distances and elevation biases
===

Import required modules
---

In [1]:
from netCDF4 import Dataset
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2
from model import Model
from GCNet import GCNet

AWS metadata
---

In [2]:
A = GCNet("site_info.nc")
A.loadMeta()
nSites = A.getNSites()

ERAI metadata
---

In [3]:
modelMetaFN = "erai_geog_sub.nc"
M = Model(modelMetaFN)
M.loadMeta()

1d dataset


Set up dataframe
---

In [4]:
df = pd.DataFrame(columns=['Site #', 'Site name', 'Site elev',\
                           'Grid Dist Closest', \
                           'Grid Elev Closest', 'Grid Elev Closest Bias', \
                           'Grid Elev Interp',  'Grid Elev Interp Bias'], \
                           index=range(nSites))

AWS info and interpolated elevation from old dataframe
---

In [5]:
dfInfoOld = pd.read_pickle('erai_site_info.pkl' )
df['Site #'] = dfInfoOld['Site #']
df['Site name'] = dfInfoOld['Site name']
df['Site elev'] = dfInfoOld['Site elev']
df['Grid Elev Interp'] = dfInfoOld['ElevCorr']
df['Grid Elev Interp Bias'] = dfInfoOld['ElevErr']

Find closest points
---

In [6]:
closestDist = []
closestElev = []
closestBias = []
for awsIX in (range(nSites)):
    awsElev = A.getElev( awsIX )
    awsLatLon = ( A.getLat( awsIX ), A.getLon( awsIX ))
    closest = M.closestPoints( awsLatLon )
    d = closest[0]
    px = d['ixLon']
    py = d['ixLat']
    pdist = d['distance']
    plat  = M.getLat2d(py,px)
    plon  = M.getLon2d(py,px)
    pelev = M.getElev(py,px)
    perr = pelev - awsElev
    L = (awsIX, awsElev, pdist, plat, plon, pelev, perr)
#     print "%d %d: M: D = %.2f km, lat/lon/elev = %.2f/%.2f/%.2f m, elev_err = %.2f m" % L
    
    closestDist.append( pdist )
    closestElev.append( pelev )
    closestBias.append( perr )

df['Grid Dist Closest'] = closestDist
df['Grid Elev Closest'] = closestElev
df['Grid Elev Closest Bias'] = closestBias

In [7]:
pd.options.display.float_format = '{:,.2f}'.format

df

,Site #,Site name,Site elev,Grid Dist Closest,Grid Elev Closest,Grid Elev Closest Bias,Grid Elev Interp,Grid Elev Interp Bias
0,1,Swiss Camp,"1,176.00",28.08,"1,057.57",-118.43,973.00,-203.00
1,2,Crawford Point1,"2,022.00",8.25,"1,845.64",-176.36,1895.42,-126.58
2,3,NASA-U,"2,334.00",23.97,"2,370.61",36.61,2319.32,-14.68
3,4,GITS,"1,869.00",33.61,"1,447.48",-421.52,1620.95,-248.05
4,5,Humboldt,"1,995.00",31.43,"2,061.75",66.75,1965.60,-29.40
5,6,Summit,"3,199.00",8.02,"3,167.76",-31.24,3171.18,-27.82
6,7,Tunu-N,"2,052.00",26.00,"2,014.45",-37.55,2077.42,25.42
7,8,DYE-2,"2,099.00",19.24,"2,110.88",11.88,2100.99,1.99
8,9,JAR1,932.00,37.41,770.82,-161.18,795.82,-136.18
9,10,Saddle,"2,467.00",36.22,"2,377.55",-89.45,2386.55,-80.45


Create a netCDF file for the new data
-----

In [8]:
newNCFN = 'dist_elev_meta_erai.nc'
try: ncfile.close()  # just to be safe, make sure dataset is not already open.
except: pass

ncfile = Dataset(newNCFN,mode='w',format='NETCDF4_CLASSIC') 
ncfile.title = "Closest-point and interpolated grid point metadata"

stn_dim = ncfile.createDimension('station', nSites)

distVar = ncfile.createVariable('distanceClosest', np.float32, ('station'))
distVar.units = "km"
distVar.description = "Distance to nearest grid point"
distVar.long_name = "Distance to nearest grid point"
distVar[:] = list( df['Grid Dist Closest'].astype("float") )

elevClosestVar = ncfile.createVariable('elevationClosest', np.float32, ('station'))
elevClosestVar.units = "m"
elevClosestVar.description = "Elevation at nearest grid point"
elevClosestVar.long_name = "Elevation at nearest grid point"
elevClosestVar[:] = list( df['Grid Elev Closest'].astype("float") )

biasClosestVar = ncfile.createVariable('elevationBiasClosest', np.float32, ('station'))
biasClosestVar.units = "m"
biasClosestVar.description = "Elevation bias at nearest grid point"
biasClosestVar.long_name = "Elevation bias at nearest grid point"
biasClosestVar[:] = list( df['Grid Elev Closest Bias'].astype("float") )

elevInterpVar = ncfile.createVariable('elevationInterp', np.float32, ('station'))
elevInterpVar.units = "m"
elevInterpVar.description = "Interpolated elevation"
elevInterpVar.long_name = "Interpolated elevation"
elevInterpVar[:] = list( df['Grid Elev Interp'].astype("float") )

biasInterpVar = ncfile.createVariable('elevationBiasInterp', np.float32, ('station'))
biasInterpVar.units = "m"
biasInterpVar.description = "Interpolated elevation bias"
biasInterpVar.long_name = "Interpolated elevation bias"
biasInterpVar[:] = list( df['Grid Elev Interp Bias'].astype("float") )

ncfile.close()